# Used car listings analysis from a P2P commerce platform

*In this project, I will work with a dataset of used cars from eBay Kleinanzeigen, a classifieds section of the German eBay website. The dataset was originaly scraped and uploaded to Kaggle. You can find it [here](https://www.kaggle.com/orgesleka/used-cars-database/data).*

The aim of this project is to clean the data and analyze the included **used car listings** using pandas library. 
![Image](https://upload.wikimedia.org/wikipedia/commons/thumb/e/ed/Pandas_logo.svg/1200px-Pandas_logo.svg.png)

## Opening and discovering the dataset

***


In [1]:
import pandas as pd
import numpy as np

autos = pd.read_csv("autos.csv", encoding = "Latin-1") 
# Need to specify the encoding type because 
# the one by default, "UTF-8", does not work.

In [2]:
autos.info()  # Let's see basic info about the dataset

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
dateCrawled            50000 non-null object
name                   50000 non-null object
seller                 50000 non-null object
offerType              50000 non-null object
price                  50000 non-null object
abtest                 50000 non-null object
vehicleType            44905 non-null object
yearOfRegistration     50000 non-null int64
gearbox                47320 non-null object
powerPS                50000 non-null int64
model                  47242 non-null object
odometer               50000 non-null object
monthOfRegistration    50000 non-null int64
fuelType               45518 non-null object
brand                  50000 non-null object
notRepairedDamage      40171 non-null object
dateCreated            50000 non-null object
nrOfPictures           50000 non-null int64
postalCode             50000 non-null int64
lastSeen               50000 non-null obj

**First observation :** it seems that few columns have null object : *"vehicleType"*, *"gearbox"*, *"model"*, *"fuelType"* and *"notRepairedDamage"*. However, none have more than ~20% null values.

In [3]:
autos.head() # Let's see the first few rows of the dataset

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,odometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


**Second observation :** the column names use *camelcase* instead of Python's preferred *snakecase*

Let's **reword some of the column names** based on the data dictionary to be more descriptive and **convert the column names** from camelcase to snakecase.

In [4]:
autos.columns # To show only the columns' names.

Index(['dateCrawled', 'name', 'seller', 'offerType', 'price', 'abtest',
       'vehicleType', 'yearOfRegistration', 'gearbox', 'powerPS', 'model',
       'odometer', 'monthOfRegistration', 'fuelType', 'brand',
       'notRepairedDamage', 'dateCreated', 'nrOfPictures', 'postalCode',
       'lastSeen'],
      dtype='object')

In [5]:
# First : rewording some of the column names

autos.rename({"yearOfRegistration":"registration_year",
              "monthOfRegistration":"registration_month",
              "notRepairedDamage" : "unrepaired_damage",
              "dateCreated":"ad_created"},
             inplace=True,axis =1)

In [6]:
# Second : rewording some of the column names

snakecase_names=[]

for i in autos.columns:
    snakecase_names.append(str(i).lower())
 
autos.columns = snakecase_names # We are changing columns attribute into our new columns names
print(autos.columns)

Index(['datecrawled', 'name', 'seller', 'offertype', 'price', 'abtest',
       'vehicletype', 'registration_year', 'gearbox', 'powerps', 'model',
       'odometer', 'registration_month', 'fueltype', 'brand',
       'unrepaired_damage', 'ad_created', 'nrofpictures', 'postalcode',
       'lastseen'],
      dtype='object')



## Basic data exploration and data cleaning
***

In [7]:
autos.describe(include="all")

,datecrawled,name,seller,offertype,price,abtest,vehicletype,registration_year,gearbox,powerps,model,odometer,registration_month,fueltype,brand,unrepaired_damage,ad_created,nrofpictures,postalcode,lastseen
count,50000,50000,50000,50000,50000,50000,44905,50000.000000,47320,50000.000000,47242,50000,50000.000000,45518,50000,40171,50000,50000.0,50000.000000,50000
unique,48213,38754,2,2,2357,2,8,NaN,2,NaN,245,13,NaN,7,40,2,76,NaN,NaN,39481
top,2016-03-11 22:38:16,Ford_Fiesta,privat,Angebot,$0,test,limousine,NaN,manuell,NaN,golf,"150,000km",NaN,benzin,volkswagen,nein,2016-04-03 00:00:00,NaN,NaN,2016-04-07 06:17:27
freq,3,78,49999,49999,1421,25756,12859,NaN,36993,NaN,4024,32424,NaN,30107,10687,35232,1946,NaN,NaN,8
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005.073280,NaN,116.355920,NaN,NaN,5.723360,NaN,NaN,NaN,NaN,0.0,50813.627300,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105.712813,NaN,209.216627,NaN,NaN,3.711984,NaN,NaN,NaN,NaN,0.0,25779.747957,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.000000,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.0,1067.000000,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1999.000000,NaN,70.000000,NaN,NaN,3.000000,NaN,NaN,NaN,NaN,0.0,30451.000000,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2003.000000,NaN,105.000000,NaN,NaN,6.000000,NaN,NaN,NaN,NaN,0.0,49577.000000,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008.000000,NaN,150.000000,NaN,NaN,9.000000,NaN,NaN,NaN,NaN,0.0,71540.000000,NaN


**Observations :**
1. columns ***price*** and ***odometer*** should be displayed as numeric data
2. columns ***registration_year***, ***powerps***, ***registration_month***, ***nrofpictures*** and ***postalcode*** seem to contain some NaN objects
3. columns ***seller***, ***offertype***, ***abtest***, ***gearbox*** and ***unrepaired_damage*** contains only two values

### 1) Cleaning of *price* and *odometer* columns

#### A) Conversion to numeric data

In [8]:
print(autos["price"].unique())
# it seems that all prices are in dollar 

['$5,000' '$8,500' '$8,990' ... '$385' '$22,200' '$16,995']


In [9]:
autos["price"] = autos["price"].str.replace("$","") # erasing currency
autos["price"] = autos["price"].str.replace(",","") # adapting numbers' format
autos["price"] = autos["price"].map(float)

# datas contained in "price" are now numeric values
print(autos["price"].head(3))


0    5000.0
1    8500.0
2    8990.0
Name: price, dtype: float64


In [10]:
print(autos["odometer"].unique())

autos["odometer"] = autos["odometer"].str.replace("km","") # erasing metric
autos["odometer"] = autos["odometer"].str.replace(",","") # adapting numbers' format
autos["odometer"] = autos["odometer"].map(float)

print("\n")
print(autos["odometer"].head(3))


['150,000km' '70,000km' '50,000km' '80,000km' '10,000km' '30,000km'
 '125,000km' '90,000km' '20,000km' '60,000km' '5,000km' '100,000km'
 '40,000km']


0    150000.0
1    150000.0
2     70000.0
Name: odometer, dtype: float64


In [11]:
# We rename columns' names to add the unit

autos.rename({"odometer":"odometer_km",
              "price":"price_dollar"},
             inplace=True,axis =1)

#### B) Cleaning of unrealistic values : *price_dollar* column

In [12]:
print(autos["price_dollar"].unique().shape)

autos["price_dollar"].describe()

(2357,)


count    5.000000e+04
mean     9.840044e+03
std      4.811044e+05
min      0.000000e+00
25%      1.100000e+03
50%      2.950000e+03
75%      7.200000e+03
max      1.000000e+08
Name: price_dollar, dtype: float64

It seems that *price_dollar* column contains unrealistic values, such as 0 and 10.000.000

In [13]:
print(autos["price_dollar"].value_counts().sort_index().head())
print("\n")
print(autos["price_dollar"].value_counts().sort_index().tail())

# We chain here some series' methods to have the frequency table 
# of the prices, sorted per values.

# We display only a few first and last values.

0.0    1421
1.0     156
2.0       3
3.0       1
5.0       2
Name: price_dollar, dtype: int64


10000000.0    1
11111111.0    2
12345678.0    3
27322222.0    1
99999999.0    1
Name: price_dollar, dtype: int64


**Observations:**

There are some values that are completely unrealistic for used car prices. People often display these kind of values when selling something online, to draw attention of potential viewers.

We need to drop these values as they prevent us from analysing deeper our dataset (caculation of mean is completely biased for example).

In [14]:
# Let's remove outliers of price_dollar column
mask_bool = (autos["price_dollar"] > 200000 ) | (autos["price_dollar"] < 500)
autos.loc[mask_bool,"price_dollar"] = np.nan

In [15]:
print(autos["price_dollar"].unique().shape)
autos["price_dollar"].describe()

(2201,)


count     45089.000000
mean       6271.147863
std        8464.802153
min         500.000000
25%        1500.000000
50%        3500.000000
75%        7900.000000
max      198000.000000
Name: price_dollar, dtype: float64

Now, mean and min/max values better reflects real used car prices. Thanks to the **unique().shape**, we can calculate that we removed around 150 prices from our dataset. 
Let's do the same analysis for our *odometer_km* column.

#### C) Cleaning of unrealistic values : *odometer_km* column

In [16]:
print(autos["odometer_km"].unique().shape)

print(autos["odometer_km"].describe())

(13,)
count     50000.000000
mean     125732.700000
std       40042.211706
min        5000.000000
25%      125000.000000
50%      150000.000000
75%      150000.000000
max      150000.000000
Name: odometer_km, dtype: float64


Our *odometer_km* column has only 14 unique values. It seems that ebay forced the seller to chose an approximate number of km for its car when posting an announce. At this stage, we don't need to remove any of the data contained in the *odometer_km* column.

### 2) Cleaning of the dates columns

Looking back at the *autos.infos()* output, it seems that the **date_crawled, last_seen**, and **ad_created** columns are all identified as string values by pandas. Because these three columns are represented as strings, we need to convert the data into a numerical representation so we can understand it quantitatively. 

The other two columns which contain dates (**registration_month** and **registration_year**) are already represented as numeric values.

Let's first understand how the values in the three string columns are formatted. These columns all represent full timestamp values, like so:

In [17]:
autos[['datecrawled','ad_created','lastseen']][0:5]

,datecrawled,ad_created,lastseen
0,2016-03-26 17:47:46,2016-03-26 00:00:00,2016-04-06 06:45:54
1,2016-04-04 13:38:56,2016-04-04 00:00:00,2016-04-06 14:45:08
2,2016-03-26 18:57:24,2016-03-26 00:00:00,2016-04-06 20:15:37
3,2016-03-12 16:58:10,2016-03-12 00:00:00,2016-03-15 03:16:28
4,2016-04-01 14:38:50,2016-04-01 00:00:00,2016-04-01 14:38:50


#### A) Calculation of the distribution of values

In [18]:
A = autos['datecrawled'].str[:10] # To take only the date values

A.value_counts(normalize=True, dropna=False).sort_index().head()
# First method is used to include missing values in the distribution and to use percentages instead of counts
# Second method is used to rank by date in ascending order (earliest to latest)
# Third method is used to display only the first (earliest) 5 values

2016-03-05    0.02538
2016-03-06    0.01394
2016-03-07    0.03596
2016-03-08    0.03330
2016-03-09    0.03322
Name: datecrawled, dtype: float64

In [19]:
autos['ad_created'].str[:10].value_counts(normalize=True, dropna=False).sort_index().head()
# We do the same thing as before but chaining all the methods used

2015-06-11    0.00002
2015-08-10    0.00002
2015-09-09    0.00002
2015-11-10    0.00002
2015-12-05    0.00002
Name: ad_created, dtype: float64

In [20]:
autos['lastseen'].str[:10].value_counts(normalize=True, dropna=False).sort_index().head()

2016-03-05    0.00108
2016-03-06    0.00442
2016-03-07    0.00536
2016-03-08    0.00760
2016-03-09    0.00986
Name: lastseen, dtype: float64

In [21]:
autos['registration_year'].describe() # This column does not need any change as datas are already numeric values

count    50000.000000
mean      2005.073280
std        105.712813
min       1000.000000
25%       1999.000000
50%       2003.000000
75%       2008.000000
max       9999.000000
Name: registration_year, dtype: float64

**Observations :**
1. Thanks to the distribution of **datecrawled** column, we can conclude that multipled cars are crawled on the same day by the crawler.
2. **registration_year** column contains some odd values:
    - The minimum value is 1000, before cars were invented
    - The maximum value is 9999, many years into the future


Because a car can't be first registered after the listing was seen, any vehicle with a registration year above 2016 is definitely inaccurate. Determining the earliest valid year is more difficult. Realistically, it could be somewhere in the first few decades of the 1900s.

Let's count the number of listings with cars that fall outside the 1900 - 2016 interval and see if it's safe to remove those rows entirely, or if we need more custom logic.

#### B) Dealing with incorrect registration year data

In [22]:
autos.loc[(autos["registration_year"] > 2016 ) | (autos["registration_year"] < 1900) , "registration_year"] = np.nan

autos["registration_year"].value_counts(normalize=True,dropna=True).head(9)

2000.0    0.069834
2005.0    0.062776
1999.0    0.062464
2004.0    0.056988
2003.0    0.056779
2006.0    0.056384
2001.0    0.056280
2002.0    0.052740
1998.0    0.051074
Name: registration_year, dtype: float64

**Observation :** around 50% of the used car listings have a registration_year between 1999 and 2007.

In [23]:
autos['registration_year'].describe()

count    48028.00000
mean      2002.80351
std          7.31085
min       1910.00000
25%       1999.00000
50%       2003.00000
75%       2008.00000
max       2016.00000
Name: registration_year, dtype: float64

**Observation :** without the unrealistic values, the mean is now lower than before.

### 3) Exploring price by brand

#### A) Basic analysis of brand column

In [24]:
autos["brand"].describe()

count          50000
unique            40
top       volkswagen
freq           10687
Name: brand, dtype: object

In [25]:
autos["brand"].notnull().value_counts()

True    50000
Name: brand, dtype: int64

In [26]:
autos["brand"].unique()

array(['peugeot', 'bmw', 'volkswagen', 'smart', 'ford', 'chrysler',
       'seat', 'renault', 'mercedes_benz', 'audi', 'sonstige_autos',
       'opel', 'mazda', 'porsche', 'mini', 'toyota', 'dacia', 'nissan',
       'jeep', 'saab', 'volvo', 'mitsubishi', 'jaguar', 'fiat', 'skoda',
       'subaru', 'kia', 'citroen', 'chevrolet', 'hyundai', 'honda',
       'daewoo', 'suzuki', 'trabant', 'land_rover', 'alfa_romeo', 'lada',
       'rover', 'daihatsu', 'lancia'], dtype=object)

In [27]:
autos["brand"].value_counts(normalize = True).head(10)

volkswagen       0.21374
opel             0.10922
bmw              0.10858
mercedes_benz    0.09468
audi             0.08566
ford             0.06958
renault          0.04808
peugeot          0.02912
fiat             0.02616
seat             0.01882
Name: brand, dtype: float64

**Observations :**
1. Brand column does not contain null values. 
2. There are 40 unique brands in the dataset.
3. Top 10 brands counts for ~75% of the total listings.

#### B)  Analysis of mean car price per brand [TOP 10 ONLY]

In [28]:
autos["brand"].value_counts().index[0:10]
# We combine .value_counts() method and .index attribute to gather the brands we want to analyse

Index(['volkswagen', 'opel', 'bmw', 'mercedes_benz', 'audi', 'ford', 'renault',
       'peugeot', 'fiat', 'seat'],
      dtype='object')

In [29]:
brand_mean_dic = {} # Starting by creating an empty dictionary

for i in autos["brand"].value_counts().index[0:10]:
    subset = autos.loc[autos["brand"] == i,"price_dollar"]
    brand_mean = subset.sum()/subset.count()
    brand_mean_dic[i] = int(brand_mean)

# Sorting the dictionary to have the highest mean prices first
for i in sorted(brand_mean_dic,key=brand_mean_dic.get, reverse = True): 
    print(i,":",brand_mean_dic[i])

audi : 9484
mercedes_benz : 8670
bmw : 8418
volkswagen : 5701
seat : 4709
ford : 4267
opel : 3348
peugeot : 3329
fiat : 3211
renault : 2760


#### C) Study of a potential link between mean prices and average mileage

Let's create a new dataframe storing both mean price and mean mileage per brand.

In [30]:
# We first create a serie from our dictionary storing mean prices per brand
bmd_series = pd.Series(brand_mean_dic)
print(bmd_series)

audi             9484
bmw              8418
fiat             3211
ford             4267
mercedes_benz    8670
opel             3348
peugeot          3329
renault          2760
seat             4709
volkswagen       5701
dtype: int64


In [31]:
# We then create a single-column dataframe from this series object
df = pd.DataFrame(bmd_series, columns=['mean_price'])
print(df)

               mean_price
audi                 9484
bmw                  8418
fiat                 3211
ford                 4267
mercedes_benz        8670
opel                 3348
peugeot              3329
renault              2760
seat                 4709
volkswagen           5701


In [32]:
# We use the loop method from the last screen to calculate the mean mileage

brand_mean_dic_km = {} # Starting by creating an empty dictionary

for i in autos["brand"].value_counts().index[0:10]:
    subset = autos.loc[autos["brand"] == i,"odometer_km"]
    brand_mean = subset.sum()/subset.count()
    brand_mean_dic_km[i] = int(brand_mean)
    
# Sorting the dictionary to have the highest mean prices first
for i in sorted(brand_mean_dic_km,key=brand_mean_dic_km.get, reverse = True): 
    print(i,":",brand_mean_dic_km[i])

bmw : 132521
mercedes_benz : 130886
audi : 129643
opel : 129298
volkswagen : 128955
renault : 128223
peugeot : 127352
ford : 124131
seat : 122061
fiat : 117037


In [33]:
# Now we convert the mean mileage dictionary to put it in our previous dataframe

bmd_km_series = pd.Series(brand_mean_dic_km) # First we transform it into a serie
df['mean_km'] = bmd_km_series # Then we agregate it into our dataframe

print(df)


               mean_price  mean_km
audi                 9484   129643
bmw                  8418   132521
fiat                 3211   117037
ford                 4267   124131
mercedes_benz        8670   130886
opel                 3348   129298
peugeot              3329   127352
renault              2760   128223
seat                 4709   122061
volkswagen           5701   128955


**Observation :** It seems that there is no real correlation between both mean_price and mean_km, as all mean mileage are roughly between 125.000 and 130.000km.